In [17]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

In [36]:
# Создаём сэмпл
n_samples = 1000

fuel_efficiency_per_hour_job = (np.random.choice(3, n_samples) + 1 )
job_time = (np.random.choice(12, n_samples) + 2 )

fuel_efficiency_per_100km = (np.random.choice(20, n_samples) + 6 )
ride_distance_km = (np.random.choice(120, n_samples) + 1 )

total_fuel_spent = ride_distance_km / 100 * fuel_efficiency_per_100km + fuel_efficiency_per_hour_job * job_time

data = pd.DataFrame({'ride_distance_km': ride_distance_km,  
                     'fuel_efficiency_per_100km': fuel_efficiency_per_100km,
                     'job_time': job_time,
                     'fuel_efficiency_per_hour_job': fuel_efficiency_per_hour_job,
                     'total_fuel_spent': total_fuel_spent})
data.head(5)

,ride_distance_km,fuel_efficiency_per_100km,job_time,fuel_efficiency_per_hour_job,total_fuel_spent
0,82,6,12,2,28.92
1,60,10,5,3,21.00
2,34,12,2,2,8.08
3,19,8,7,2,15.52
4,2,7,13,1,13.14


Моделируем датасет по общему расходу топлива строительной техники исходя из двух основных процессов:
-движение техники до объекта выполнения работ
-непосредственное выполнение техникой работ

Выбраны следующие признаки:
-расход топлива на час работ
-время выполнения работ
-расход топлива при движении (на 100 км)
-длина пути до объекта выполнения работ и обратно

Предполагаемая (используемая) модель зависимости следующая:
total_fuel_spent =
ride_distance_km / 100 * fuel_efficiency_per_100km + fuel_efficiency_per_hour_job * job_time

Стоит выделить, что в предлагаемом гипотетическом варианте зависимость количества топлива на дорогу или на работу в целом определяется не просто суммой-комбинацией факторов расхода и времени/дистанции, а их умножением.

In [37]:
from sklearn.metrics import mean_absolute_error

X = data[['ride_distance_km', 'fuel_efficiency_per_100km', 'job_time', 'fuel_efficiency_per_hour_job']]
y = data['total_fuel_spent']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['ride_distance_km', 'fuel_efficiency_per_100km',
                                'job_time', 'fuel_efficiency_per_hour_job']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [0.15238188 0.65317709 1.97080627 7.43662873]
Bias: -24.646104708569126
Error: 2.7369594323830295


Попробуем построить модель линейной регрессии на базе четырех основных признаков.
Модель пытается подобрать такие веса каждого признака, СУММА которых даст наименьшее квадратичное отклонение (ошибку) при попытке предсказать/повторить результат набора данных. 

Как результат получаем определенное предсказание весов, отклонение и ошибку модели. 

In [38]:
# Создаем новый признак
data['fuel_per_ride'] = data['ride_distance_km'] * data['fuel_efficiency_per_100km']
data['fuel_per_job'] = data['job_time'] * data['fuel_efficiency_per_hour_job']
data.head(5)

,ride_distance_km,fuel_efficiency_per_100km,job_time,fuel_efficiency_per_hour_job,total_fuel_spent,fuel_per_ride,fuel_per_job
0,82,6,12,2,28.92,492,24
1,60,10,5,3,21.00,600,15
2,34,12,2,2,8.08,408,4
3,19,8,7,2,15.52,152,14
4,2,7,13,1,13.14,14,13


Пробуем сократить количество используемых признаков и одновременно "убрать" зависимость от умножения двух параметров путем создания отдельного расчетного столбца в датафрейме - fuel_per_ride; fuel_per_job

In [41]:
X = data[['fuel_per_ride','fuel_per_job']]
y = data['total_fuel_spent']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['fuel_per_ride','fuel_per_job']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [0.01 1.  ]
Bias: 1.4210854715202004e-14
Error: 6.181277711903022e-15


После выполнения указанных манипуляций повторяем обучение модели линейной регрессии. Получили результат со значительно меньшими (идеальными) сдвигом и ошибкой, так как модель полностью предсказала/повторила нашу синтетическую формулу для создания сэмпла данных.